# Eye tracking: preprocessing

This script imports the raw eye tracking (ET) data from the `Subjects` folder
and runs a few preprocessing steps on them:

1. Transform the gaze vectors into spherical coordinates (`theta`: horizontal 
gaze angle, `phi`: vertical)  
2. Calculate the time relative to the stimulus onset event.  
3. Correct for the temporal offset between the actual eye movement and the time 
the eye tracker provides the accordinmg sample (using cross-correlation with the 
instantaneous EOG signal)
4. Subtract the mean baseline in a window of `200 ms` before stimulus onset from
each trial (separately for `theta` and `phi`).
5. Optional (see parameter `blinks_interpolate` at the beginning of the notebook): identify blinks in the data and replace them via linear interpolation. 
6. Save the files as pandas data frames.  
  
⚠️ We only have ET data for a subset of the participants. 

In [1]:
import os
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from vr2f.staticinfo import COLORS, PATHS
from vr2f.eyetracking import lag_calculator_et_vs_eog, et_preprocessing

In [2]:
paths = PATHS()
colors = COLORS()

blinks_interpolate = True

In [90]:
sub_list_str = os.listdir(paths.DATA_SUBJECTS)
sub_list_str = sorted(sub_list_str)
n_valid_subs = 0

for sub_id in sub_list_str:
    print(f"Processing {sub_id}...")

    markerlog_stimonsets = et_preprocessing.get_stimonset_df(sub_id)
    if markerlog_stimonsets is None:
        print(f"    Skipping {sub_id}.")
        continue
    markerlog_stimonsets_np = markerlog_stimonsets["timestamp_corrected"].to_numpy()


    et_files, path_et_files = et_preprocessing.get_et_rawfiles(sub_id)
    if et_files is None:
        print(f"    Skipping {sub_id}.")
        continue

    df_all = pd.DataFrame()
    for f in et_files:
        t_num = int(f.split(".csv")[0][-3:])
        fname = Path(path_et_files, f)
        df = pd.read_csv(fname, sep=",")

        df = df.reset_index(drop=False)

        # skip this file if it is empty apart from the header
        if df.shape[0] == 0:
            continue
        # add 3 columns theta, phi, r to df
        df[["theta", "phi", "r"]] = et_preprocessing.cart2sph_custom(
            df["direction_x_local"], df["direction_y_local"], df["direction_z_local"]
        )

        # calculate relative time to stimulus onset:
        times = df["time"] - markerlog_stimonsets_np[t_num - 1]
        # find the index of the stimulus onset
        idx_stimonset = np.argmin(np.abs(times))

        df["times"] = (df["timestamp_lsl"] - df.loc[idx_stimonset, "timestamp_lsl"]) 

        # crop to [-0.5; 1.5]s window; assuming a stable srate of 120Hz
        srate = 120
        df = df.iloc[np.max([idx_stimonset - int(0.5 * srate), 0]) : idx_stimonset + int(1.5 * srate) + 1, :]

        # add info about the trial
        df["trial_num"] = t_num
        df["marker"] = markerlog_stimonsets["annotation"].iloc[t_num - 1]
        # split marker on whitespace and keep only last part
        df["marker"] = df["marker"].str.split().str[-1]
        # marker is a 3digit int; split it into its single digits and put each digit into a separate column
        df["viewcond"] = df["marker"].str[0].astype(int)
        df["avatar_id"] = df["marker"].str[1].astype(int)
        df["emotion"] = df["marker"].str[2].astype(int)
        df["emotion"] = df["emotion"].map(cond_dict["emotion"])
        df["avatar_id"] = df["avatar_id"].map(cond_dict["avatar_id"])
        df["viewcond"] = df["viewcond"].map(cond_dict["viewcond"])

        df_all = pd.concat([df_all, df], ignore_index=True)

    if df.shape[0] == 0:
        print(f"    Skipping {sub_id} because there is no eye tracking data.")
        continue

    # calculate a baseline for each trial
    baselines = (
        df_all.query("times < 0 & times > -0.2").loc[:, ["theta", "phi", "trial_num"]].groupby(["trial_num"]).mean()
    )

    # subtract the baseline from each trial
    df_all = df_all.merge(baselines, on="trial_num", suffixes=("", "_baseline"))
    df_all["theta"] = df_all["theta"] - df_all["theta_baseline"]
    df_all["phi"] = df_all["phi"] - df_all["phi_baseline"]

    # adaptively correct for the lag between ET and EOG
    lag_calculator = lag_calculator_et_vs_eog.LagCalculatorEyetrackingVsEog()
    lag = lag_calculator.get_et_vs_eog_lag(sub_id, df_all, plot_it=False, write_csv=True)
    df_all["times"] = df_all["times"] - lag
    print(f"    Correcting for lag of {lag}ms for {sub_id}.")

    if blinks_interpolate:
        df_all = df_all.groupby(["trial_num"]).apply(et_preprocessing.interpolate_blinks)

    # save to csv
    path_out = Path(paths.DATA_ET_PREPROC)
    if blinks_interpolate:
        fname = Path(path_out, f"{sub_id}-ET-withoutblinks.csv")
    else:
        fname = Path(path_out, f"{sub_id}-ET-withblinks.csv")
    df_all.to_csv(fname, sep=",", index=False)

    n_trials = df_all["trial_num"].nunique()
    print(f"    {sub_id}: {n_trials} trials")
    n_valid_subs += 1
    

print(f"\nN of subjects with valid ET data: {n_valid_subs}.")

Processing VR2FEM_S01...
    Correcting for lag of 0.058333333333333334ms for VR2FEM_S01.
    VR2FEM_S01: 744 trials
Processing VR2FEM_S02...
    Correcting for lag of 0.058333333333333334ms for VR2FEM_S02.
    VR2FEM_S02: 744 trials
Processing VR2FEM_S03...
Skipping VR2FEM_S03 because there are no eye tracking files.
    Skipping VR2FEM_S03.
Processing VR2FEM_S04...
    No markerlog for VR2FEM_S04
    Skipping VR2FEM_S04.
Processing VR2FEM_S05...
    Correcting for lag of 0.058333333333333334ms for VR2FEM_S05.
    VR2FEM_S05: 744 trials
Processing VR2FEM_S06...
    Skipping VR2FEM_S06 because there is no eye tracking data.
Processing VR2FEM_S07...
    Correcting for lag of 0.0625ms for VR2FEM_S07.
    VR2FEM_S07: 744 trials
Processing VR2FEM_S08...
    Correcting for lag of 0.0625ms for VR2FEM_S08.
    VR2FEM_S08: 744 trials
Processing VR2FEM_S10...
    Correcting for lag of 0.058333333333333334ms for VR2FEM_S10.
    VR2FEM_S10: 744 trials
Processing VR2FEM_S11...
    Correcting for l